In [1]:
import json
import pandas as pd
import urllib.request
import urllib.parse
import urllib.error
import os
from IPython.display import Image
import numpy as np
import cv2
import math

## Import DataSet

In [3]:
df_2dFFT = pd.read_csv("/home/kannika/codes_AI/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold.csv")
print("Fold:", set(df_2dFFT['fold']))
print(df_2dFFT.shape)
df_2dFFT

Fold: {1, 2, 3, 4, 5, 6}
(538, 12)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
...,...,...,...,...,...,...,...,...,...,...,...,...
533,533,533,NBL23_D0_30HZ_20XINF_UWELL_25660221_150600_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_150600,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0
534,534,534,NBL23_D0_30HZ_20XINF_UWELL_25660221_150802_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_150802,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0
535,535,535,NBL23_D0_30HZ_20XINF_UWELL_25660221_151001_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_151001,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0
536,536,536,NBL23_D0_30HZ_20XINF_UWELL_25660221_151246_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_151246,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0


### Split Class in each Fold

In [7]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    lst_fold.sort()
    for j in range(len(lst_fold)):
        df_classes = DataSet[DataSet["fold"]==lst_fold[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        lst_folder = list(set(df_classes["fold"]))
        print(f"[INFO]: [{j+1}].  Fold {lst_fold[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 6-Fold With Class:")
        lst_cls = list(set(df_classes['subclass']))
        print(f"Number of Classes {len(lst_cls)} Class ==> {lst_cls}")
        for m in lst_cls:
            df_f =  df_classes[df_classes["subclass"]==m]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]:  Classse {m} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*100)
    
    return

In [6]:
_(df_2dFFT)

[INFO]: [1].  Fold 1 With Shape ==> 92 Folder, (92, 12)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Normal', 'No_Splenectomy', 'Splenectomy']
[INFO]:  Classse Normal ==> With 32 Folder, 32 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 95 Folder, (95, 12)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Normal', 'No_Splenectomy', 'Splenectomy']
[INFO]:  Classse Normal ==> With 47 Folder, 47 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Classse Splenectomy ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 88 Folder, (88, 12)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Normal', 'No_Splenectomy', 'Splenectomy']
[INFO]:  Classse Normal ==> With 34 Folder, 34 images!! ==> 3 PerSon!!
[INFO]:  Cla

- Class: Normal น้อยสุด 24 ภาพ 

- Class: เป็นโรคต้องเท่ากับ 24 ภาพ (แบ่งเป็น ตัดม้าม 12 ภาพ, ไม่ตัดม้าม 12 ภาพ)

In [19]:
import random

DF_dataset = pd.DataFrame(columns=list(df_2dFFT.columns))
lst_fold = list(set(df_2dFFT["fold"]))
lst_fold.sort()
base_n = 24
# classe_lst  = ['Normal', 'No_Splenectomy', 'Splenectomy']
# lst_cls = list(set(df_classes['subclass']))
for m in range(len(lst_fold)):
    print(f"[INFO]: On Process fold-{lst_fold[m]}")
    df_fold =  df_2dFFT[df_2dFFT["fold"]==lst_fold[m]]
    lst_cls = list(set(df_fold['subclass']))
    lst_cls.sort()
    for cls in lst_cls:
        print(f"==> Split data in class : {cls} <==")
        df_cls = df_fold[df_fold["subclass"]==cls]
        if cls == "Normal":
            random_cls = df_cls.sample(base_n, replace=False)
            print(random_cls.shape[0], "imgs")
            DF_dataset = DF_dataset.append(random_cls)
        else:
            random_cls = df_cls.sample(int(base_n/2), replace=False)
            print(random_cls.shape[0], "imgs")
            DF_dataset = DF_dataset.append(random_cls)
    print("="*100)

[INFO]: On Process fold-1
==> Split data in class : No_Splenectomy <==
12 imgs
==> Split data in class : Normal <==
24 imgs
==> Split data in class : Splenectomy <==
12 imgs
[INFO]: On Process fold-2
==> Split data in class : No_Splenectomy <==
12 imgs
==> Split data in class : Normal <==
24 imgs
==> Split data in class : Splenectomy <==
12 imgs
[INFO]: On Process fold-3
==> Split data in class : No_Splenectomy <==
12 imgs
==> Split data in class : Normal <==
24 imgs
==> Split data in class : Splenectomy <==
12 imgs
[INFO]: On Process fold-4
==> Split data in class : No_Splenectomy <==
12 imgs
==> Split data in class : Normal <==
24 imgs
==> Split data in class : Splenectomy <==
12 imgs
[INFO]: On Process fold-5
==> Split data in class : No_Splenectomy <==
12 imgs
==> Split data in class : Normal <==
24 imgs
==> Split data in class : Splenectomy <==
12 imgs
[INFO]: On Process fold-6
==> Split data in class : No_Splenectomy <==
12 imgs
==> Split data in class : Normal <==
24 imgs
==> Sp

/tmp/ipykernel_90016/2813824669.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_dataset = DF_dataset.append(random_cls)
/tmp/ipykernel_90016/2813824669.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_dataset = DF_dataset.append(random_cls)
/tmp/ipykernel_90016/2813824669.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_dataset = DF_dataset.append(random_cls)
/tmp/ipykernel_90016/2813824669.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_dataset = DF_dataset.append(random_cls)
/tmp/ipykernel_90016/2813824669.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fut

In [21]:
DF_dataset = DF_dataset.reset_index(drop=True)
print("Fold:", set(DF_dataset['fold']))
print(DF_dataset.shape)
DF_dataset.head()

Fold: {1, 2, 3, 4, 5, 6}
(288, 12)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,237,237,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
1,234,234,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
2,324,324,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1
3,268,268,HN38_D0_30HZ_20XINF_UWELL_25660223_145146_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN38_D0_30HZ_20XINF_UWELL_25660223_145146,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
4,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1


#### #️⃣ Check dataset

In [22]:
_(DF_dataset)

[INFO]: [1].  Fold 1 With Shape ==> 48 Folder, (48, 12)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Normal', 'No_Splenectomy', 'Splenectomy']
[INFO]:  Classse Normal ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 48 Folder, (48, 12)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Normal', 'No_Splenectomy', 'Splenectomy']
[INFO]:  Classse Normal ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 48 Folder, (48, 12)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Normal', 'No_Splenectomy', 'Splenectomy']
[INFO]:  Classse Normal ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Cla

In [23]:
DF_dataset.to_csv("/home/kannika/codes_AI/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold-splitclass.csv")